In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [9]:
k = [2, 3, 4, 5, *range(10, 101, 10)]
q = 100

In [18]:
def gen_data(k, q):
  pts = np.random.uniform(0, 1, size=(q, k))
  og = np.zeros(k)
  for i in range(0, q):
    while np.allclose(pts[i], og):
      pts[i] = np.random.uniform(0, 1, size=k)
  return pts